In [1]:
!pip install -U langchain langchain-community langchain-huggingface pypdf sentence-transformers faiss-cpu rank_bm25


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.4/483.4 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 65.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.8 MB/s eta 0:00:00:00:0100:01
 

# Setup

## HF Token

In [ ]:
import os

# WARNING: Insecure for public notebooks.
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Please put HF_TOKEN"

## Model Config

In [3]:
# Update this path for your Kaggle environment
PDF_PATH = "/kaggle/input/Zerodha_varsity.pdf" 
EMBEDDING_MODEL_NAME = "BAAI/bge-large-en-v1.5"
RERANKER_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"
LLM_REPO_ID = "meta-llama/Meta-Llama-3-8B-Instruct"

# Loading Document and Chunking

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Loading
loader = PyPDFLoader(PDF_PATH)
docs = loader.load()
print(f"Loaded {len(docs)} pages from PDF.")

# Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
chunks = text_splitter.split_documents(docs)
print(f"Split document into {len(chunks)} chunks.")

Loaded 1909 pages from PDF.
Split document into 3710 chunks.


# Advanced Retriever Setup

## Rertievers Setup

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# Embedding Model
embedding_function = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
vectorstore = FAISS.from_documents(chunks, embedding_function)

# Keyword Matching Model
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k = 10

# Combining both into a Hybrid Retriever
hybrid_retriever = EnsembleRetriever(
    retrievers=[keyword_retriever, vector_retriever],
    weights=[0.5, 0.5],
    c = 10 # 'k' in Reciprocal Rank Fusion algorithm
)
print("Hybrid retriever created.")

2025-09-04 13:10:56.288741: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756991456.453119      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756991456.502796      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Hybrid retriever created.


In [15]:
hybrid_retriever.invoke("IPO")

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2021-11-29T10:51:08+04:00', 'source': '/kaggle/input/Zerodha_varsity.pdf', 'total_pages': 1909, 'page': 108, 'page_label': '109'}, page_content='CHAPTER 1 4 \n \nSupplementary  Note \n \nIPO, OFS, and FPO – How are they different?  \nIPO \nInitial Public Offering is w hen a company is introduced in to the publicly traded stock \nmarkets for the very first time. In the IPO, the promoters of the company choose to offer \na certain percentage of shares to the public. The reason for going public and the \nprocess of an IPO is exp lained in detail in Chapter 4 and 5.  \nThe primary reason for going public is to raise capital which would be to fund expansion \nprojects or cash out early investors. After the IPO is listed on the exchange and is traded \nin the secondary market, promoters of the company might still want additional capital \nfor which there are three options available: Rights Issue, Off

## Re-Ranking setup

In [16]:
# --- 4.3. Setup Reranking ---
reranker_model = HuggingFaceCrossEncoder(model_name=RERANKER_MODEL_NAME) # reranks all chunks retrieved by hybrid_retriever
compressor = CrossEncoderReranker(model=reranker_model, top_n=5) # takes top 5 chunks, drops others

compression_retriever = ContextualCompressionRetriever( # Needs base retriever(FAISS + BM25) and compressor(Reranker)
    base_compressor=compressor,
    base_retriever=hybrid_retriever
)
print("Reranking compression retriever created.")

# compression_retriever(hybrid_retriever(FAISS, BM25), compressor(BAAI, n=5))

Reranking compression retriever created.


In [17]:
compression_retriever.invoke("IPO")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(id='7ebb6c76-747c-491e-af33-73bcffc6b300', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2021-11-29T10:51:08+04:00', 'source': '/kaggle/input/Zerodha_varsity.pdf', 'total_pages': 1909, 'page': 35, 'page_label': '36'}, page_content='18.The valuation of the company increases as and when the business , revenues and \nprofitability increases\n19.An IPO is a process by means of which a company can raise fund. The funds raised can be for  \nany valid reason – for CAPEX, restructuring debt, rewarding shareholders etc\n31 zerodha.com/varsity\n∏'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2021-11-29T10:51:08+04:00', 'source': '/kaggle/input/Zerodha_varsity.pdf', 'total_pages': 1909, 'page': 108, 'page_label': '109'}, page_content='CHAPTER 1 4 \n \nSupplementary  Note \n \nIPO, OFS, and FPO – How are they different?  \nIPO \nInitial Public Offering is w hen a company is introduced in to the publicl

# Initializing Model and RAG Pipeline

In [19]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain_huggingface.chat_models import ChatHuggingFace
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

# Initializing the base LLM from Hugging Face
endpoint_llm = HuggingFaceEndpoint(
    repo_id=LLM_REPO_ID, task="text-generation", max_new_tokens=512,
    top_k=50, top_p=0.9, temperature=0.6
)
# Wrap it in the ChatHuggingFace adapter
llm = ChatHuggingFace(llm=endpoint_llm)
print(f"Chat LLM '{LLM_REPO_ID}' initialized.")

Chat LLM 'meta-llama/Meta-Llama-3-8B-Instruct' initialized.


## Rephrasing question for Memory

In [20]:
# Prompt for rephrasing the input question into including past chats
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# This chain will rephrase the question and then retrieve documents
history_aware_retriever = create_history_aware_retriever(
    llm, compression_retriever, contextualize_q_prompt
)

# compression_retriever(hybrid_retriever(FAISS, BM25), compressor(BAAI, n=5))
# history_aware_retriever(Llama, compression retriever, prompt)

## Final Q/A chain formation

In [21]:
# New system prompt for the final answer generation
qa_system_prompt = (
    "You are an expert financial assistant. Use the following retrieved context to answer the user's question accurately. "
    "If the information is not in the context, say that you cannot find the answer in the provided documents. "
    "Be concise and helpful."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Re-create the document chain with the new prompt
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# This is your final, conversational RAG chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
print("Conversational RAG chain created successfully.")

# compression_retriever(hybrid_retriever(FAISS, BM25), compressor(BAAI, n=5)) -> we get 5 chunks (ranked)
# history_aware_retriever(Llama, compression retriever, contextualize_qa_prompt) -> 5 chunks + query_rephrased
# question_answer_chain(Llama, qa_prompt) -> chat_history + query_rephrased
# rag_chain(history_aware_retriever, question_answer_chain) -> we get final output

Conversational RAG chain created successfully.


# TESTING

In [22]:
from langchain_core.messages import HumanMessage, AIMessage
def ask_with_memory(query, chat_history):
    response = rag_chain.invoke({"input": query, "chat_history": chat_history})
    print("\nAnswer:")
    print(response["answer"])

    # Update history
    chat_history.extend([HumanMessage(content=query), AIMessage(content=response["answer"])])
    print("="*100)
    print("\nSOURCES:")
    for i, doc in enumerate(response["context"]):
        print(f"\t Source {i+1} (Page {doc.metadata.get('page', 'N/A')}):\n \"{doc.page_content[:100]}...\"")
        print("_"*100)

    print("#"*50, "END", "#"*50)

In [23]:
from langchain_core.messages import HumanMessage, AIMessage

# To store the conversation
chat_history = []

query = str(input("Query: "))
while(query!= 'STOP'):
    ask_with_memory(query, chat_history)
    query = str(input("Query: "))

Query:  What are IPOs


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Answer:
According to the provided context, an IPO (Initial Public Offering) is when a company is introduced into the publicly traded stock markets for the first time. In an IPO, the company's promoters choose to offer a certain percentage of shares to the public. The primary reason for going public and the process of an IPO is explained in detail in Chapters 4 and 5. The main purposes of an IPO include:

1. Raising capital to fund expansion projects
2. Allowing early investors to cash out
3. Rewarding employees
4. Gaining visibility for the company

The context also mentions that merchant bankers act as key partners with the company during the IPO process, and SEBI regulates the IPO market.

SOURCES:
	 Source 1 (Page 35):
 "18.The valuation of the company increases as and when the business , revenues and 
profitability inc..."
____________________________________________________________________________________________________
	 Source 2 (Page 108):
 "CHAPTER 1 4 
 
Supplementary  Note

Query:  Tell me more about it


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Answer:
Based on the provided context, here are more details about IPOs:

1. **Reasons for going public**: Companies go public to raise funds for their capital expenditure (Capex) requirements, to avoid debt and the associated finance charges, and to spread the risk among a large group of people.

2. **IPO process**: The IPO process involves several steps:
	* Appointing a merchant banker (also called a Book Running Lead Manager (BRLM)/Lead Manager (LM)) to assist the company with various aspects of the IPO process.
	* Conducting a due diligence on the company by the merchant banker to ensure legal compliance and issuing a due diligence certificate.
	* Preparing the listing documents, including the Draft Red Herring Prospectus (DRHP), under the guidance of the merchant banker.
	* Underwriting shares by the merchant banker, where they agree to buy all or part of the IPO shares and resell them to the public.
	* Filing the DRHP with SEBI and obtaining a nod from SEBI to proceed with the I

Query:  Elaborate 1st point more


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Answer:
The first point I mentioned was:

**Reasons for going public**: Companies go public to raise funds for their capital expenditure (Capex) requirements, to avoid debt and the associated finance charges, and to spread the risk among a large group of people.

Let me elaborate on each of these reasons:

1. **Raising funds for Capex requirements**: Companies often require large amounts of capital to fund their expansion plans, research and development, and other capital expenditure projects. Going public allows them to raise funds from a large number of investors, which can be used to finance these projects.

2. **Avoiding debt and finance charges**: By going public, companies can avoid taking on debt to raise funds. This means they don't have to pay interest on loans, which can save them a significant amount of money. Additionally, they don't have to worry about the associated finance charges, which can eat into their profits.

3. **Spreading risk among a large group of people**: W

Query:  STOP


In [31]:
chat_history[3] # Sample element in 'chat_history'

AIMessage(content='Based on the provided context, here are more details about IPOs:\n\n1. **Reasons for going public**: Companies go public to raise funds for their capital expenditure (Capex) requirements, to avoid debt and the associated finance charges, and to spread the risk among a large group of people.\n\n2. **IPO process**: The IPO process involves several steps:\n\t* Appointing a merchant banker (also called a Book Running Lead Manager (BRLM)/Lead Manager (LM)) to assist the company with various aspects of the IPO process.\n\t* Conducting a due diligence on the company by the merchant banker to ensure legal compliance and issuing a due diligence certificate.\n\t* Preparing the listing documents, including the Draft Red Herring Prospectus (DRHP), under the guidance of the merchant banker.\n\t* Underwriting shares by the merchant banker, where they agree to buy all or part of the IPO shares and resell them to the public.\n\t* Filing the DRHP with SEBI and obtaining a nod from SE